# MakiFlow usage examples

This notebook contains basic guidelines of using MakiFlow. MakiFlow is not a serious framework like PyTorch or TensorFlow, this is an instrument I built in order to improve quality and speed of my researches in the field of Neural Networks, especially CNNs. If you found this library useful and have some thoughs how to improve it or what to fix I'd like to listen to you. My email: igor.kilbas@mail.ru.

## Importing MakiFlow

As far as I don't know how to create high quality libraries, you're gonna have some pain. Here is the way I import MakiFlow on my computer:

In [1]:
import sys
sys.path.append('/home/student401/MakiFlow')

## Creating CNN based on MakiFlow

Now you have an access to MakiFlow and can import stuff related to it.
In order to create CNN you need to defined layers order first. Here is what I mean by that:

In [2]:
from makiflow.layers import ConvLayer, DenseLayer, MaxPoolLayer, FlattenLayer

layers = [
        ConvLayer(kw=3, kh=3, in_f=3, out_f=64, name='input_layer'),
        ConvLayer(kw=3, kh=3, in_f=64, out_f=64, name=2),
        MaxPoolLayer(),
    
        ConvLayer(kw=3, kh=3, in_f=64, out_f=128, name=4),
        ConvLayer(kw=3, kh=3, in_f=128, out_f=128, name=5),
        MaxPoolLayer(),
    
        ConvLayer(kw=3, kh=3, in_f=128, out_f=256, name=6),
        ConvLayer(kw=3, kh=3, in_f=256, out_f=256, name=7),
        MaxPoolLayer(),
    
        ConvLayer(kw=3, kh=3, in_f=256, out_f=512, name=8),
        ConvLayer(kw=3, kh=3, in_f=512, out_f=512, name=9),
        ConvLayer(kw=3, kh=3, in_f=512, out_f=512, name=10),
        MaxPoolLayer(),
    
        FlattenLayer(),
        DenseLayer(input_shape=2048, output_shape=1024, name=12),
        DenseLayer(input_shape=1024, output_shape=1024, name=13),
        DenseLayer(input_shape=1024, output_shape=10, activation=None, name='out_put_layer')
    # The last layer always is gonna have no activation function! Just always pass None into 'activation' argument!
    ]

/home/student401/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


We have built basic VGG model. As you might have noticed, you have to directly pass the number of input feature maps and output feature maps, so be careful with that. Tip: number of the input feature maps in the next layer equals number of the feature maps in the previous layer. Anyway, TensorFlow will get you know if something is wrong.

To create CNN, we need ConvModel class:

In [3]:
from makiflow import ConvModel

model = ConvModel(layers=layers, input_shape=[64, 32, 32, 3], output_shape=[64, 10], name='My_MakiFlow_little_VGG')

That's it! You've just created your first CNN based on MakiFlow! But we still have some stuff to do.

First, ConvModel requires TensorFlow Session in order to work. You can treat ConvModel as a car and Session as an engine. Let's add an engine to our car:

In [4]:
import tensorflow as tf

session = tf.Session()
model.set_session(session)

## Training CNN

Now we can use the model. Let's train it. We'll use cifar10 dataset for this purpose.

In [5]:
import numpy as np
import keras
from keras.datasets import cifar10

(Xtrain, Ytrain), (Xtest, Ytest) = cifar10.load_data()
    
Xtrain = Xtrain.astype(np.float32)
Xtest = Xtest.astype(np.float32)

Xtrain /= 255
Xtest /= 255

# Convert class vectors to binary class matrices.
Ytrain = keras.utils.to_categorical(Ytrain, 10)
Ytest = keras.utils.to_categorical(Ytest, 10)

Using TensorFlow backend.


Second step is to define our training parameters:
learning_rate;
optimizer - ConvModel uses TensorFlow optimizers in order to train the model;
epochs;

We will use RMSProp for training the model.

In [6]:
epochs = 2
lr = 1e-5*128
epsilon = 1e-6
optimizer = tf.train.RMSPropOptimizer(learning_rate=lr, epsilon=epsilon)

In order to train the model we can call two methods:
verbose_fit - tests the model on train data and test data;
pure_fit - tests the model only on test data, it works much faster than verbose_fit.

We will use pure fit.

In [7]:
info = model.pure_fit(Xtrain, Ytrain, Xtest, Ytest, optimizer=optimizer, epochs=epochs)

  0%|          | 0/781 [00:00<?, ?it/s]

Epoch: 0 Train accuracy: 0.21233446193512973 Train cost: 2.0322607538318254 Test accuracy 0.25739999999999996 Test cost 1.9103252704326923


100%|██████████| 781/781 [00:12<00:00, 61.89it/s]


Epoch: 1 Train accuracy: 0.4534793300054287 Train cost: 1.4788230974867071 Test accuracy 0.4505 Test cost 1.4869333902994792


pure_fit method returns dictionary with tests' info on each epoch.

In [8]:
info

{'train costs': [2.0322607538318254, 1.4788230974867071],
 'train errors': [0.7876655380648703, 0.5465206699945713],
 'test costs': [1.9103252704326923, 1.4869333902994792],
 'test errors': [0.7426, 0.5495]}

## Evaluating the model

In order to evaluate the model, simply call the method evaluate:

In [9]:
model.evaluate(Xtest, Ytest)

100%|██████████| 156/156 [00:01<00:00, 147.34it/s]

Accuracy: 0.4505 Cost: 1.4869339580719287


## Making predictions using MakiFlow model

In order to make predictions, we use predict method. 
This method requires tensor of shape (number_of_samples, dim, dim, channels).
It always returns numpy array with probabilities of belonging this particular sample(or samples) to each class.

In [15]:
model.predict(Xtest[0:1])

array([[0.01108474, 0.00861162, 0.12099435, 0.36020404, 0.06199497,
        0.27742323, 0.10490333, 0.03577648, 0.01214113, 0.00686607]],
      dtype=float32)

# Saving/loading the model

Since we have trained model, we would like to save it's weights and architecture. ConvModel class has methods written for this purpose: to_json(), save_weights().

## Saving weights

Keep in your mind that you have to insert @FULL PATH of the directory you want to save weights in + name of the weights file@. Weights saving system uses TensorFlow's checkpoint file, so be careful and don't forget to add '.ckpt' at the end of the name of the weights file. Here is the way I do this:

In [10]:
model.save_weights('/home/student401/temp/my_model_weights.ckpt')

Model saved to /home/student401/temp/my_model_weights.ckpt


## Saving architecture

As might have noticed, model's architecture is gonna be stored in json file. In order to save the architecture, you need to pass in @PATH to directory you want to save the architecture in + name of the json file@. In this case it's not necessary to insert FULL PATH.

In [11]:
model.to_json('/home/student401/temp/my_model_architecture.json')

Model's architecture is saved to /home/student401/temp/my_model_architecture.json.


## Creating model from existing architecure file

It's always good to have pretrained models so that you can use their weights or fine-tune them at any time. Let's create new model with the architecture we defined erlier.
In order to do this, we need to import Builder class that creates ConvModel from json file. Builder class has one main method: convmodel_from_json(). It takes path to json file with the architecture we need and returns ConvModel object with predefined architecture.

In [13]:
from makiflow.save_recover.builder import Builder

new_model = Builder.convmodel_from_json('/home/student401/temp/my_model_architecture.json')

## Loading weights

In order to load weights, simply call load_weights() method in ConvModel object. This method takes path to the checkpoint file and loads weights. You also must initialize the model with some session, otherwise it won't be able to load weights.
#### IMPORTANT!
CNN (ConvModel) must have the same architecture like model thess weights were taken from, all the layer names must be the same, otherwise it may cause errors!

In [15]:
new_model.set_session(session)
new_model.load_weights('/home/student401/temp/my_model_weights.ckpt')

INFO:tensorflow:Restoring parameters from /home/student401/temp/my_model_weights.ckpt
Model restored


Let's evaluate the model to make sure if it's the same as previous.

In [16]:
new_model.evaluate(Xtest, Ytest)

100%|██████████| 156/156 [00:01<00:00, 142.99it/s]

Accuracy: 0.4505 Cost: 1.4869339580719287


As you can see, it is.